In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
from scipy import optimize
from scipy.stats import sem

import os
import sys

import matplotlib.pyplot as plt
import matplotlib.ticker
from matplotlib.patches import Rectangle
from matplotlib.colors import ListedColormap, BoundaryNorm

from venn import venn, pseudovenn
import seaborn as sns

from Bio import SeqIO, SeqUtils

from addict import Dict

import math

In [6]:
sys.path.append("../")
import plotting as my_plot

sys.path.append("./mean_field_elect/")
import proteomics_functions as prot_fun

# High-risk HCPs 

In [44]:
# Generate the high-risk candidate list
path = './lit_data/biophorum_high_risk_clean.csv'
df_lit = pd.read_csv(path)
df_master = pd.read_csv('./generated_tables/location_master_with_only_mean_properties.csv')
prot_fun.get_name_matches(df_master, path, 'high_risk')
df_risk = df_master[df_master.high_risk_name.notnull()].copy()
df_risk.reset_index(drop=True, inplace=True)

for i, cont in df_risk.iterrows():
    for col in ['impact', 'impact_type', 'function']:
        data = df_lit.loc[df_lit.desc_lower_2 == cont.high_risk_name, col].iloc[0] 
        df_risk.at[i, col] = data
        
df_risk.to_csv('./generated_tables/high_risk_unreviewed.csv', index=False)

# Difficult to remove

In [5]:
# df_master = pd.read_csv('./generated_tables/location_master_with_properties.csv')

# df_master.drop(columns=['pI_0', 'mass_0', 'net_charge_0', 'net_charge_neg_0', 'net_charge_pos_0',
#                        'charge_dens_C_m2_0', 'charge_dens_neg_C_m2_0',
#                        'charge_dens_pos_C_m2_0', 'pI_1', 'mass_1',
#                        'net_charge_1', 'net_charge_neg_1', 'net_charge_pos_1',
#                        'charge_dens_C_m2_1', 'charge_dens_neg_C_m2_1',
#                        'charge_dens_pos_C_m2_1', 'pI_2', 'mass_2',
#                        'net_charge_2', 'net_charge_neg_2', 'net_charge_pos_2',
#                        'charge_dens_C_m2_2', 'charge_dens_neg_C_m2_2',
#                        'charge_dens_pos_C_m2_2', 'cysteine_cont_percent_0', 'cysteine_num_0',
#                        'cysteine_cont_percent_1', 'cysteine_num_1', 'cysteine_cont_percent_2',
#                        'cysteine_num_2'], inplace=True)

In [6]:
# prot_fun.get_name_matches(df_master, './lit_data/biophorum_clean.csv', 'Biophorum DTR', name_append='')
# prot_fun.get_name_matches(df_master, './lit_data/biophorum_high_risk_clean.csv', 'Biophorum high-risk', name_append='')
# prot_fun.get_name_matches(df_master, './lit_data/levy_2014_clean.csv', 'Levy 2014 - Association', name_append='')
# prot_fun.get_name_matches(df_master, './lit_data/zhang_table_2_clean.csv', 'Zhang 2016 - Pro A', name_append='')
# prot_fun.get_name_matches(df_master, './lit_data/levy_2016_clean.csv', 'Levy 2016 - Coelution', name_append='')
# prot_fun.get_name_matches(df_master, './lit_data/molden_clean.csv', 'Molden 2021 - 29 DS', name_append='')
# prot_fun.get_name_matches(df_master, './lit_data/falkenberg_table_4_clean.csv', 'Falkenberg 2019 - 6 DS', name_append='')
# prot_fun.get_name_matches(df_master, './lit_data/kreimer_table_1_clean.csv', 'Kreimer 2017 - 1 DS', name_append='')

In [67]:
# papers = ['Biophorum DTR', 'Biophorum high-risk', 'Levy 2014 - Association', 'Zhang 2016 - Pro A', 
#           'Levy 2016 - Coelution', 'Molden 2021 - 29 DS', 'Falkenberg 2019 - 6 DS', 'Kreimer 2017 - 1 DS']

# df_dtr = df_master[(df_master[papers[0]].notnull()) |
#                    (df_master[papers[1]].notnull()) |
#                    (df_master[papers[2]].notnull()) |
#                    (df_master[papers[3]].notnull()) |
#                    (df_master[papers[4]].notnull()) |
#                    (df_master[papers[5]].notnull()) |
#                    (df_master[papers[6]].notnull()) |
#                    (df_master[papers[7]].notnull()) ]

# df_dtr.reset_index(inplace=True, drop=True)
# df_dtr.to_csv('./generated_tables/difficult_to_remove_unreviewed_1.csv', index=False)

## Addition of Aboulaich data

In [138]:
df_master = pd.read_csv('./generated_tables/location_master_with_properties.csv')
df_master.drop(columns=['pI_0', 'mass_0', 'net_charge_0', 'net_charge_neg_0', 'net_charge_pos_0',
                       'charge_dens_C_m2_0', 'charge_dens_neg_C_m2_0',
                       'charge_dens_pos_C_m2_0', 'pI_1', 'mass_1',
                       'net_charge_1', 'net_charge_neg_1', 'net_charge_pos_1',
                       'charge_dens_C_m2_1', 'charge_dens_neg_C_m2_1',
                       'charge_dens_pos_C_m2_1', 'pI_2', 'mass_2',
                       'net_charge_2', 'net_charge_neg_2', 'net_charge_pos_2',
                       'charge_dens_C_m2_2', 'charge_dens_neg_C_m2_2',
                       'charge_dens_pos_C_m2_2', 'cysteine_cont_percent_0', 'cysteine_num_0',
                       'cysteine_cont_percent_1', 'cysteine_num_1', 'cysteine_cont_percent_2',
                       'cysteine_num_2'], inplace=True)

ab_name = 'Aboulaich 2014 - Association'
prot_fun.get_name_matches(df_master, './lit_data/aboulaich_table_1_clean.csv', ab_name, name_append='')
df_ab = df_master[df_master['Aboulaich 2014 - Association'].notnull()].copy()
df_ab.set_index('accession_0', inplace=True)

In [139]:
df_dtr = pd.read_csv('./generated_tables/difficult_to_remove_reviewed_1.csv')
df_dtr.set_index('accession_0', inplace=True)

In [141]:
shared = list(set(df_dtr.index).intersection(df_ab.index))
new = [accn for accn in df_ab.index if accn not in shared]

for accn in shared:
    df_dtr.at[accn, ab_name] = df_ab.at[accn, ab_name]
    
df_dtr_new = pd.concat([df_dtr, df_ab.loc[new]])
df_dtr_new.to_csv('./generated_tables/difficult_to_remove_unreviewed_2.csv')